In [1]:
import ROOT
import pandas as pd
import numpy as np
import os
import sys
import ipynbname
from pathlib import Path

project_root = str(ipynbname.path().parent.parent)
sys.path.append(project_root)

from processing import SiphraAcquisition, fit_peak_expbg

# ROOT.gROOT.SetBatch(False)
# ROOT.gEnv.SetValue("Canvas.ShowEventStatus", 1)
# ROOT.gEnv.SetValue("Canvas.ShowToolBar", 1)
# ROOT.gEnv.SetValue("Canvas.UseGL", 0)
# ROOT.gROOT.ProcessLine("gVirtualX = new TGX11();")


In [2]:
# Constants
BITS12 = 2**12
BITS9 = 2**9 # 512 typical number of bins used
# Energy emission peaks in MeV
K40_MEV = 1.460
TL208_MEV = 2.614

In [3]:
BG_PMchs58 = []
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/1_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=2,
                                      exposure_sec=119.634,
                                      sipm_chs="5-8"))
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/2_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=6,
                                      exposure_sec=161.062,
                                      sipm_chs="5-8"))
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/3_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=10,
                                      exposure_sec=148.920,
                                      sipm_chs="5-8"))
BG_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/4_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=14,
                                      exposure_sec=164.828,
                                      sipm_chs="5-8"))

BG_PMchs14 = []
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/5_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Trh1_cmisvoffset127_Background.csv',
                                      active_chs=2,
                                      exposure_sec=34.836,
                                      sipm_chs="1-4"))
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/6_SiPM_ChannelsTest_Ch1-4_Ch6_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=6,
                                      exposure_sec=80.792,
                                      sipm_chs="1-4"))
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/7_SiPM_ChannelsTest_Ch1-4_Ch10_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=10,
                                      exposure_sec=54.386,
                                      sipm_chs="1-4"))
BG_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/8_SiPM_ChannelsTest_Ch1-4_Ch14_CT_Thr1_cmisvoffset127_Background.csv',
                                      active_chs=14,
                                      exposure_sec=90.837,
                                      sipm_chs="1-4"))

Cs137_PMchs14 = []
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/12_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=2,
                                      exposure_sec=23.381,
                                      sipm_chs="1-4"))
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/11_SiPM_ChannelsTest_Ch1-4_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=6,
                                      exposure_sec=24.606,
                                      sipm_chs="1-4"))
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/10_SiPM_ChannelsTest_Ch1-4_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=10,
                                      exposure_sec=24.144,
                                      sipm_chs="1-4"))
Cs137_PMchs14.append(SiphraAcquisition(project_root+'/data/260209/9_SiPM_ChannelsTest_Ch1-4_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=14,
                                      exposure_sec=24.645,
                                      sipm_chs="1-4"))

Cs137_PMchs58 = []
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/13_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=2,
                                      exposure_sec=24.720,
                                      sipm_chs="5-8"))
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/14_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=6,
                                      exposure_sec=24.916,
                                      sipm_chs="5-8"))
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/15_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=10,
                                      exposure_sec=24.848,
                                      sipm_chs="5-8"))
Cs137_PMchs58.append(SiphraAcquisition(project_root+'/data/260209/16_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv',
                                      active_chs=14,
                                      exposure_sec=24.934,
                                      sipm_chs="5-8"))

datasets_lsts = [BG_PMchs58, Cs137_PMchs14, BG_PMchs14, Cs137_PMchs58]
colors = [ROOT.kRed, ROOT.kSpring, ROOT.kBlue, ROOT.kOrange, ROOT.kCyan, ROOT.kYellow, ROOT.kGreen, ROOT.kViolet]

In [4]:
datasets_lsts[0][0].get_active_chs_data()

array([823., 278., 370., ..., 258., 308., 345.], shape=(99901,))

In [19]:
if ROOT.gROOT.FindObject("canvas"):
    ROOT.gROOT.FindObject("canvas").Close()
canvas = ROOT.TCanvas("canvas", "canvas", 800, 600)
ROOT.gStyle.SetOptStat(11)
ROOT.gStyle.SetStatFontSize(0.03)
ROOT.gStyle.SetStatW(0.15)
legend_ary = ROOT.TLegend(0.65, 0.7, 0.8, 0.88)

Yinit = 0.82 # For stat boxes
hists_58BG = []
for acq, color in zip(datasets_lsts[0], colors):
    hist = ROOT.TH1F(f"h58bgCh{acq.active_chs[0]}", "", BITS9, 0, BITS12)
    hist.Fill(acq.get_active_chs_data())
    hist.Scale(1/acq.exposure)
    # Preeting up...
    hist.GetXaxis().SetTitle("ADC channel number")
    hist.GetYaxis().SetTitle(r"Count rate (s^{-1})")
    hist.SetLineColor(color+2)
    legend_ary.SetHeader('SIPHRA channel')
    legend_ary.AddEntry(hist, f"Ch. {acq.active_chs[0]}", 'l')
    hists_58BG.append(hist)
    hists_58BG[-1].Draw("sames hist")
canvas.SetLogy()

ROOT.gPad.Update()
Yinit = 0.82
for i, sp in enumerate(hists_58BG):
    st = sp.FindObject("stats")
    print(type(st))
    st.SetY1NDC(Yinit - i*0.08)
    st.SetY2NDC(0.1)


legend_ary.Draw()
canvas.Draw()
ROOT.gPad.Modified()
ROOT.gPad.Update()

<class cppyy.gbl.TPaveStats at 0x5ad362632c50>
<class cppyy.gbl.TPaveStats at 0x5ad362632c50>
<class cppyy.gbl.TPaveStats at 0x5ad362632c50>
<class cppyy.gbl.TPaveStats at 0x5ad362632c50>


Warning in <TROOT::Append>: Replacing existing TH1: h58bgCh14 (Potential memory leak).


In [6]:
hists_58BG